In [1]:
from helper import create_batches_test
import tensorflow as tf
import numpy as np
import shelve
import joblib
import random
import time
import math
import matplotlib.pyplot as plt

#Cuz the file is inside 'code' directory
mount_point = "../shelved_data/"

with shelve.open(mount_point+'IAM_Data') as shelf:
    vocabulary = shelf['chars']
#     list_of_images = shelf['list_of_images']
#     image_labels = shelf['image_labels']

with shelve.open(mount_point+'Test/image_data') as shelf:
    list_of_images = shelf['list_of_images']

image_arrays = joblib.load(mount_point+'Test/image_arrays')

#List_images ko sort karo
#list_of_images.sort()

#Convert vocabulary to list
vocabulary = list(vocabulary)
#Sort so as to have the same ordering every time..
vocabulary.sort()
vocabulary.append("<Blank>")

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## Test Parameters

In [2]:
img_height = 104
img_width = 688
vocab_size = len(vocabulary)


conv_out_height, conv_out_width = (int(math.ceil(img_height/(2**3 * 3))),int(math.ceil(img_width/(2**3 * 3))))

#Number of time_steps to unroll for..
seq_len = conv_out_height * conv_out_width

training_list = list_of_images
batch_size = len(training_list)

#random.seed(100)
#random.shuffle(training_list)

In [3]:
#batches_x,batches_y = create_batches(batch_size,training_list,image_arrays,image_labels,vocabulary)
#print(len(batches_x),len(batches_y))
# list_of_images.sort()
new_training_list = list_of_images
batches_x = create_batches_test(batch_size,new_training_list,image_arrays)

In [9]:
batches_x[0].shape

(104, 688, 128)

## Predict using Model

In [4]:
resume_epoch = 1725

In [5]:
outputs = []

In [6]:
with tf.Session() as sess:
    # Load the graph
    saver = tf.train.import_meta_graph('../model/200_5_Lines_RNN_'+str(resume_epoch)+'.meta')
    # Restore the weights and biases
    saver.restore(sess, '../model/200_5_Lines_RNN_'+str(resume_epoch))

    #Extract the placeholders
    inputs = sess.graph.get_tensor_by_name('Placeholder:0')
    target_indices = sess.graph.get_tensor_by_name('targets/indices:0')
    target_values = sess.graph.get_tensor_by_name('targets/values:0')
    target_shape = sess.graph.get_tensor_by_name('targets/shape:0')
    
    time_steps = sess.graph.get_tensor_by_name('Placeholder_1:0')
    dropout_lstm = sess.graph.get_tensor_by_name('Placeholder_2:0')
    dropout_fc = sess.graph.get_tensor_by_name('Placeholder_3:0')
    
    decoded = sess.graph.get_tensor_by_name('CTCGreedyDecoder:1')
    cost = sess.graph.get_tensor_by_name('Mean:0')
    label_error_rate = sess.graph.get_tensor_by_name('Mean_1:0')
    
    start_time = time.time()

    for b in range(len(batches_x)):
        feed = {inputs:batches_x[b].transpose([2,0,1]),
                time_steps:np.array([seq_len]*batch_size),dropout_lstm:1.0, dropout_fc:1.0,
               }

        d = sess.run(decoded, feed_dict=feed)

        outputs.append(d)

        end_time = time.time()   
    
        time_taken = end_time - start_time

        print("{:.2f}\n".format(time_taken))

INFO:tensorflow:Restoring parameters from ../model/200_5_Lines_RNN_1725


InternalError: Dst tensor is not initialized.
	 [[Node: _arg_Placeholder_0_0/_205 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_708__arg_Placeholder_0_0", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: transpose/_221 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_627_transpose", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
training_list.index('otsu_11.jpg')

In [ ]:
for out in outputs:
    content = "".join([vocabulary[char] for char in out])
    print(content)
    print('----------\n')

In [ ]:
plt.imshow(batches_x[0].transpose([2,0,1])[123],cmap='gray')